In [1]:
from scripts.experiments.symmetries import pauli_reduce, clifford_symmetry, get_coupled_qudits_by_gate
from scripts.experiments.symmetries.src.clifford import clifford_phase_decomposition
from scripts.experiments.symmetries.src.block_decomposition import block_decompose, ordered_block_sizes
from sympleq.models.Ising import ising_chain_hamiltonian, ising_2d_hamiltonian, heuristic_clifford_symmetry
from sympleq.core.circuits.gate_decomposition_to_circuit import gate_to_circuit
from sympleq.core.circuits import Gate, Circuit, gate_to_circuit
import numpy as np

In [2]:

N = 6
J = 1
h = 0.5
H = ising_chain_hamiltonian(N, J, h, periodic=True)


In [3]:
F = heuristic_clifford_symmetry(N)
# print(F.symplectic)
S, T = block_decompose(F.symplectic, 2)

h_S, h_T = clifford_phase_decomposition(F.symplectic, F.phase_vector, S, T, int(H.lcm))
S_gate = Gate('S', F.qudit_indices, S, F.dimensions, h_S)
T_gate = Gate('T', F.qudit_indices, T, F.dimensions, h_T)

assert F == Circuit(F.dimensions, [T_gate.inv(), S_gate, T_gate]).composite_gate()

assert H.to_standard_form() == F.act(H).to_standard_form()
assert T_gate.act(S_gate.act(T_gate.inv().act(H))).to_standard_form() == H.to_standard_form()

assert S_gate.act(T_gate.inv().act(H)).to_standard_form() == T_gate.inv().act(H).to_standard_form()

print('Got T and S')
print('Qubit cost is ', max(get_coupled_qudits_by_gate(S_gate)))

Got T and S
Qubit cost is  2.0


In [4]:

### Test F unitary

C_F = gate_to_circuit(F)
# C_S = gate_to_circuit(S)
# C_T = gate_to_circuit(T)
assert C_F.act(H).to_standard_form() == H.to_standard_form()

U_F = C_F.unitary().toarray()
H_hilbert = H.to_hilbert_space().toarray()
assert np.all(np.abs(U_F @ H_hilbert @ U_F.conj().T - H_hilbert) < 1e-8)

print('F passed')
### Test S unitary

H_prime = T_gate.act(H)

C_S = gate_to_circuit(S_gate)

U_S = C_S.unitary().toarray()

H_hilbert = H.to_hilbert_space().toarray()

U_T = gate_to_circuit(T_gate).unitary().toarray()

H_prime_hilbert = U_T @ H_hilbert @ U_T.conj().T


lhs = U_S @ H_prime_hilbert @ U_S.conj().T
rhs = H_prime_hilbert

assert np.all(np.abs(lhs.conj() - rhs) < 1e-8), f'\n{np.around(lhs, 3)} \n{np.around(rhs, 3)}'

print(H_prime)  # .to_hilbert_space().toarray()
np.set_printoptions(threshold = np.inf)
np.set_printoptions(linewidth = np.inf)

print(S_gate.symplectic)


F passed


AssertionError: 
[[-0. +0.j  0.5-1.j  0.5-1.j ... -0. +0.j  0. -0.j  0. +0.j]
 [ 0.5+1.j  0. +0.j  0. -0.j ... -0. +0.j  0. -0.j  0. -0.j]
 [ 0.5+1.j  0. +0.j -0. +0.j ...  0. +0.j  0. +0.j  0. +0.j]
 ...
 [-0. -0.j -0. +0.j  0. -0.j ... -0. +0.j -0. +0.j  0.5+1.j]
 [ 0. +0.j  0. +0.j  0. +0.j ... -0. +0.j -0. -0.j  0.5+1.j]
 [ 0. -0.j  0. +0.j -0. -0.j ...  0.5-1.j  0.5-1.j -0. -0.j]] 
[[-0. +0.j  0.5-0.j  0.5-1.j ... -0. -0.j -0. -0.j -0. +0.j]
 [ 0.5+0.j -0. -0.j  0. +0.j ... -0. +0.j -0. -0.j -0. +0.j]
 [ 0.5+1.j  0. -0.j  0. -0.j ... -0. -0.j  0. -0.j  0. +0.j]
 ...
 [-0. +0.j -0. -0.j -0. +0.j ... -0. -0.j  0. -0.j  0.5+1.j]
 [-0. +0.j -0. -0.j  0. +0.j ... -0. -0.j  0. -0.j  0.5+0.j]
 [-0. +0.j -0. -0.j -0. -0.j ...  0.5-1.j  0.5-0.j  0. -0.j]]

In [5]:
C_np = Circuit(S_gate.dimensions, C_S[0:-1])
C_S = gate_to_circuit(S_gate)

print(C_np)


H [0]
H [1]
H [2]
H [3]
H [4]
H [5]
H [0]
SUM [1 0]
H [0]
H [2]
SUM [3 2]
H [2]
H [4]
SUM [5 4]
H [4]
H_inv [0]
H_inv [1]
H_inv [2]
H_inv [3]
H_inv [4]
H_inv [5]
SWAP [1 0]
SWAP [3 2]
SWAP [5 4]

